# Land cover classification at the Mississppi Delta

In this notebook, you will use a k-means **unsupervised** clustering
algorithm to group pixels by similar spectral signatures. **k-means** is
an **exploratory** method for finding patterns in data. Because it is
unsupervised, you don’t need any training data for the model. You also
can’t measure how well it “performs” because the clusters will not
correspond to any particular land cover class. However, we expect at
least some of the clusters to be identifiable as different types of land
cover.

You will use the [harmonized Sentinel/Landsat multispectral
dataset](https://lpdaac.usgs.gov/documents/1698/HLS_User_Guide_V2.pdf).
You can access the data with an [Earthdata
account](https://www.earthdata.nasa.gov/learn/get-started) and the
[`earthaccess` library from
NSIDC](https://github.com/nsidc/earthaccess):

## STEP 1: Set up

### Step 1a: Load libraries and set GDAL parameters

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Import all libraries you will need for this analysis</li>
<li>Configure GDAL parameters to help avoid connection errors:
<code>python      os.environ["GDAL_HTTP_MAX_RETRY"] = "5"      os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"</code></li>
</ol></div></div>

In [1]:
### Import libraries

# Create reproducible file paths
import os

# Serializing and unserializing objects (save objects to disk and load later)
import pickle

# Regular expressions
import re

# Give warnings
import warnings

# Project coordinate systems for spatial data and mapping
import cartopy.crs as ccrs

# Access satellite imagery through the NASA API
import earthaccess

# Spatial data analysis
import earthpy as et

# Work with vector/shapefiles
import geopandas as gpd

# Vizualization tools
import geoviews as gv
import hvplot.pandas
import hvplot.xarray

# Work with arrays
import numpy as np

# Work with tabular data
import pandas as pd

# Work with raster data
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import xarray as xr

# Use progress bar
from tqdm.notebook import tqdm

# Polygons
from shapely.geometry import Polygon

# Use kmeans clustering
from sklearn.cluster import KMeans

### set GDAL (Geospatial Data Abstraction Library) parameters
os.environ["GDAL_HTTP_MAX_RETRY"] = "5"
os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"

### don't show non-critical warnings
warnings.simplefilter('ignore')

c:\Users\nymve\miniconda3\envs\earth-analytics-python\Lib\site-packages\earthpy\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


### Step 1b: Run the caching decorator

Below you can find code for a caching **decorator** which you can use in
your code. To use the decorator:

``` python
@cached(key, override)
def do_something(*args, **kwargs):
    ...
    return item_to_cache
```

This decorator will **pickle** the results of running the
`do_something()` function, and only run the code if the results don’t
already exist. To override the caching, for example temporarily after
making changes to your code, set `override=True`. Note that to use the
caching decorator, you must write your own function to perform each
task!

You might notice that typically in these assignments, we start by creating a data_dir to store our data files. Here, our caching decorator is making the data directory for us.

In [2]:
### make the caching decorator
def cached(func_key, override=False): # Add =True if wanting to rerun
    """
    A decorator to cache function results
    
    Parameters
    ==========
    key: str
      File basename used to save pickled results
    override: bool
      When True, re-compute even if the results are already stored
    """
    def compute_and_cache_decorator(compute_function):
        """
        Wrap the caching function
        
        Parameters
        ==========
        compute_function: function
          The function to run and cache results
        """
        def compute_and_cache(*args, **kwargs):
            """
            Perform a computation and cache, or load cached result.
            
            Parameters
            ==========
            args
              Positional arguments for the compute function
            kwargs
              Keyword arguments for the compute function
            """
            ### Add an identifier from the particular function call
            if 'cache_key' in kwargs:
                key = '_'.join((func_key, kwargs['cache_key']))
            else:
                key = func_key

            ### define a file path based on the directory structure in earthpy
            path = os.path.join(
                
                ### earthpy directory
                et.io.HOME, 
                
                ### earthpy dataset
                et.io.DATA_NAME, 
                
                ### make a subdirectory called "jars"
                'jars', 
                
                ### use f-string (formatted string) to create a string by embedding the value
                ### of the variable "key" into the string 
                ### use .pickle file extension (a pickle file is a serialized python objecT)
                f'{key}.pickle')
            
            ### Check if the cache exists already or if we should override caching
            if not os.path.exists(path) or override:
                
                ### Make jars directory if needed
                os.makedirs(os.path.dirname(path), exist_ok=True)
                
                ### Run the compute function as the user did
                result = compute_function(*args, **kwargs)
                
                ### Pickle the object (save to file)
                ### open the file at filename (wb - write binary)
                with open(path, 'wb') as file:
                    
                    ### save the result without needing to recompute when loading
                    ### it back into Python
                    pickle.dump(result, file)
            
            ### if the file already exists/we are not overriding the cache
            else:
               
                ### Unpickle the object (load the cached result) (rb - read binary)
                with open(path, 'rb') as file:
                    
                    ### use pickle.load to unserialize the file back into a python object
                    result = pickle.load(file)
                    
            return result
        
        return compute_and_cache
    
    return compute_and_cache_decorator

## STEP 2: Study site

For this analysis, you will use a watershed from the [Water Boundary
Dataset](https://www.usgs.gov/national-hydrography/access-national-hydrography-products),
HU12 watersheds (WBDHU12.shp).

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Download the Water Boundary Dataset for region 8 (Mississippi)</li>
<li>Select watershed 080902030506</li>
<li>Generate a site map of the watershed</li>
</ol>
<p>Try to use the <strong>caching decorator</strong></p></div></div>

We chose this watershed because it covers parts of New Orleans an is
near the Mississippi Delta. Deltas are boundary areas between the land
and the ocean, and as a result tend to contain a rich variety of
different land cover and land use types.

In [5]:
### assign the hydrologic unit code
HUC_LEVEL = 12

# Download, unzip, and read shapefile, using cache decorator
@cached(f'wbd_08_hu{HUC_LEVEL}_gdf')

# Make a function to read in the file
def read_wbd_file(wbd_filename, cache_key):

    # Define the URL we're pulling data from
    wbd_url = (

        ### add base URL
        "https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/WBD/HU2/Shape/"

        # Insert the name of the specific file we want
        f"{wbd_filename}.zip"
    )

    # Download data and unzip it into the directory
    wbd_dir = et.data.get_data(url = wbd_url)

    # Path to shapefile in the fir
    wbd_path = os.path.join(wbd_dir,
                            'Shape',                            
                            f'WBDHU{HUC_LEVEL}.shp')
    
    # Read the shp as gdf
    wbd_gdf = gpd.read_file(wbd_path,
                            
                            # Use pyogrio library
                            engine = 'pyogrio')
    
    # Give us the gdf for the watershed boundary
    return wbd_gdf

In [ ]:
### download, unzip, and read the shapefile, and use the caching decorator to store it:

### use the cached decorator to wrap the function we'll make (read_wbd_file) 

### define a function (read_wbd_file()) to download the watershed boundary data


    ### define the URL to download data


        ### insert the name of the specific file we want using an f-string


    ### download the data and unzip it into the directory we defined earlier


    ### make path to the shapefile in the directory


                            ### create a subfolder


                            ### make the shapefile name


    ### read the shapefile as a gdf


                            ### use pyogrio library to read the shapefile 
                            ### (better performance with large data)


    ### return the gdf of the watershed boundaries



In [ ]:
# open the shapefile using the read_wbd_file function that we created
wbd_gdf = read_wbd_file("WBD_08_HU2_Shape",
                        f'hu{HUC_LEVEL}')

Extracted output to C:\Users\nymve\earth-analytics\data\earthpy-downloads\WBD_08_HU2_Shape


In [ ]:
# Look at data
wbd_gdf

,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,states,...,name,hutype,humod,tohuc,noncontrib,noncontr_1,shape_Leng,shape_Area,ObjectID,geometry
0,{8AFB1AF9-7296-4303-89DE-14CD073B859A},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"535297,540579",29441.81,119.15,LA,...,Gourd Bayou-Youngs Bayou,S,"LE,ID,DD",080500011308,0.0,0.0,NaN,NaN,1,"POLYGON ((-92.00021 32.53586, -91.99994 32.535..."
1,{916A17A6-B4A0-4FD7-9BB8-FFD1936B15B2},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,535512,11406.67,46.16,LA,...,Hams Creek,S,ID,080802050104,0.0,0.0,NaN,NaN,2,"POLYGON ((-93.37574 30.58982, -93.3747 30.5891..."
2,{493C7EC1-2F1C-4B84-AFFB-6F6868A9868E},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"547190,559640",29138.21,117.92,LA,...,Caney Creek-Bayou D'Arbonne,S,NM,080402060503,0.0,0.0,NaN,NaN,3,"POLYGON ((-93.07761 32.88752, -93.07784 32.887..."
3,{49A3C087-B460-4F97-9D99-78CBB675248B},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"77417,78285",17759.39,71.87,AR,...,L'Aigle Creek-Saline River,S,NM,080402020206,0.0,0.0,NaN,NaN,4,"POLYGON ((-92.08947 33.29383, -92.0897 33.2938..."
4,{0FB41498-11EA-4AB1-AF05-E2A8E5E2E274},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,1628466,98564.62,398.88,LA,...,West Cote Blanche Bay,W,NM,080801030800,0.0,0.0,NaN,NaN,5,"POLYGON ((-91.62408 29.73947, -91.62195 29.737..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636,{9E524E78-2605-48CB-A41F-618AFCDF513D},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"695171,692611",9132.46,36.96,MS,...,Widow Bayou-Big Sunflower River,S,"AW,LE,TF",080302071707,0.0,0.0,NaN,NaN,2637,"POLYGON ((-90.76273 32.97428, -90.76209 32.973..."
2637,{C11913D9-C534-4755-884C-4CAD470ED143},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"77280,81842",20587.50,83.31,AR,...,Lindermans Lake-Bayou Des Arc,S,LE,080203010306,0.0,0.0,NaN,NaN,2638,"POLYGON ((-91.73427 34.99197, -91.7342 34.9923..."
2638,{3EEBF422-01AC-4322-A63C-24C0A34E1E4F},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,714675,21405.46,86.62,MO,...,Brewer Lake,S,"LE,DD,IT,TF",080103000102,0.0,0.0,NaN,NaN,2639,"POLYGON ((-89.13715 36.97285, -89.13387 36.970..."
2639,{4734715C-0F4A-4211-BBAE-86605B20B79A},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"547104,553857",38848.09,157.21,LA,...,Blounts Creek-Calcasieu River,S,ID,080802030302,0.0,0.0,NaN,NaN,2640,"POLYGON ((-92.75965 31.12593, -92.75881 31.125..."


In [9]:
### Filter the shapefile to the specific watershed we're using

# Define the gdf for the watershed by subsetting the gdf of the whole watershed dataset
delta_gdf = wbd_gdf[wbd_gdf[
                
    # Filter the gdf to the row(s) with the watershe we want             
    f'huc{HUC_LEVEL}'].isin(['080902030506'])].dissolve() 

    
    ### use "dissolve" to merge the geometries of all the rows matching the target watershed


### check it out
delta_gdf

,geometry,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,...,huc12,name,hutype,humod,tohuc,noncontrib,noncontr_1,shape_Leng,shape_Area,ObjectID
0,"POLYGON ((-89.97047 29.74687, -89.96593 29.750...",{E942B72E-599E-48F5-908A-EA5265701C14},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"536881,539539",37355.86,151.17,...,080902030506,Manuel Canal-Spanish Lake,D,GC,080902030508,0.0,0.0,NaN,NaN,2560


In [12]:
### Make a site map with satellite imagery in the background
(
    # Project the delta_gdf to Mercator
    delta_gdf.to_crs(ccrs.Mercator())

    # Use hvplot
    .hvplot(

        # Make the watershed transparent
        alpha = 0.40, fill_color = "white",

        # Add satellite basemap
        tiles = 'EsriImagery',

        crs = ccrs.Mercator())

        # Set plot size
        .opts(width = 600, height = 300)
)


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-response"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div></div><div class="callout-body-container callout-body"><p>Write a 2-3 sentence <strong>site description</strong> (with
citations) of this area that helps to put your analysis in context.</p></div></div>


**YOUR SITE DESCRIPTION HERE**

## STEP 3: Multispectral data

### Step 3a: Search for data

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Log in to the <code>earthaccess</code> service using your Earthdata
credentials:
<code>python      earthaccess.login(persist=True)</code></li>
<li>Modify the following sample code to search for granules of the
HLSL30 product overlapping the watershed boundary from May to October
2023 (there should be 76 granules):
<code>python      results = earthaccess.search_data(          short_name="...",          cloud_hosted=True,          bounding_box=tuple(gdf.total_bounds),          temporal=("...", "..."),      )</code></li>
</ol></div></div>

In [5]:
### Log in to earthaccess


In [ ]:
### Search for HLS granules we want


    ### specify which dataset and spatial resolution we want 


    ### specify that we're using cloud data


    ### use the bounding box from our watershed boundary


    ### set the temporal range of the data

### Step 3b: Compile information about each granule

I recommend building a GeoDataFrame, as this will allow you to plot the
granules you are downloading and make sure they line up with your
shapefile. You could also use a DataFrame, dictionary, or a custom
object to store this information.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>For each search result:
<ol type="1">
<li>Get the following information (HINT: look at the [‘umm’] values for
each search result):
<ul>
<li>granule id (UR)</li>
<li>datetime</li>
<li>geometry (HINT: check out the shapely.geometry.Polygon class to
convert points to a Polygon)</li>
</ul></li>
<li>Open the granule files. I recommend opening one granule at a time,
e.g. with (<code>earthaccess.open([result]</code>).</li>
<li>For each file (band), get the following information:
<ul>
<li>file handler returned from <code>earthaccess.open()</code></li>
<li>tile id</li>
<li>band number</li>
</ul></li>
</ol></li>
<li>Compile all the information you collected into a GeoDataFrame</li>
</ol></div></div>

In [7]:
### make a function to process all the granules from the earthaccess search
### and extract information for each granule

### define the function


    ### make and display a progress bar


    ### use a regular expression to extract tile_id and bank from .tif files


    ### accumulate gdf rows from each granule


    ### accumulate into url df


    ### loop over granules to extract info


        ### locate metadata (UMM = universal metadata model)


        ### pull out unique identifier for the granule


        ### extract date/time 


        ### extact boundary coordinates for granule


        ### make polygon using coordinate points for granule


        ### get url and open granule


        ### loop through each file in the granule


            ### use url regular expression to get url


            ### if match is found, append data to link_rows gdf we initialized


                    ### makes a gdf with the granule's data and geometry
         

        ### update progress bar after each granule is done


    ### combine into a single gdf   


    ### return the final gdf file


### Step 3c: Open, crop, and mask data

This will be the most resource-intensive step. I recommend caching your
results using the `cached` decorator or by writing your own caching
code. I also recommend testing this step with one or two dates before
running the full computation.

This code should include at least one **function** including a
numpy-style docstring. A good place to start would be a function for
opening a single masked raster, applying the appropriate scale
parameter, and cropping.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>For each granule:
<ol type="1">
<li><p>Open the Fmask band, crop, and compute a quality mask for the
granule. You can use the following code as a starting point, making sure
that <code>mask_bits</code> contains the quality bits you want to
consider: ```python # Expand into a new dimension of binary bits bits =
( np.unpackbits(da.astype(np.uint8), bitorder=‘little’)
.reshape(da.shape + (-1,)) )</p>
<p># Select the required bits and check if any are flagged mask =
np.prod(bits[…, mask_bits]==0, axis=-1) ```</p></li>
<li><p>For each band that starts with ‘B’:</p>
<ol type="1">
<li>Open the band, crop, and apply the scale factor</li>
<li>Name the DataArray after the band using the <code>.name</code>
attribute</li>
<li>Apply the cloud mask using the <code>.where()</code> method</li>
<li>Store the DataArray in your data structure (e.g. adding a
GeoDataFrame column with the DataArray in it. Note that you will need to
remove the rows for unused bands)</li>
</ol></li>
</ol></li>
</ol></div></div>

In [9]:
### apply cached decorator to function



### write function that computes reflectance data using 
### search results (df of urls) and watershed boundary


    ### write a function to open raster from url, apply scale factor, crop, and mask data

    

    ### write function to apply a cloud mask



        ### return the mask

    

    ### grab metadata


    ### loop through each image and its metadata

        
        ### loop over each datetime/tile_id combination



        ### open granule cloud cover

        
        
        
        ### get cloud mask band based on url


        ### compute cloud mask


        ### loop through each spectral band to open, crop, and mask the band

                

                ### add the DataArray to the metadata df

                
                ### append the row to granule_da_rows


            ### reassemble the metadata df



In [ ]:
### apply the function


In [ ]:
### check out the dataframe


### Step 3d: Merge and Composite Data

You will notice for this watershed that:   
1. The raster data for each date are spread across 4 granules  
2. Any given image is incomplete because of clouds

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">

*   1. For each band:  
    *   a. For each date:  
        *   i. Merge all 4 granules  
        *   ii. Mask any negative values created by interpolating from the nodata value of -9999 (`rioxarray`) should account for this, but doesn't appear to when merging. If you leave these values in, they will create problems later on
    *   b. Concatenate the merged DataArrays along a new date dimension  
    *   c. Take the mean in the date dimension to create a composite image that fills cloud gaps  
    *   d. Add the band as a dimensions, and give the DataArray a name  
*   2. Concatenate along the band dimension


In [11]:
### apply cache decorator


### create a function to merge and composite reflectance data from multiple granules
### end result: single, composite reflectance image for each spectral band


    ### initialize a list to store dfs
    

    ### initialize a list to store composites after procesing
    

    ### loop over each spectral band



        ### loop over date/time of image acquisition and merge granules for each data

           
            ### mask negative values (could be no data or invalid data)
            
            

            ### append to merged_das list we initialized
            
            
        ### composite images across dates


        
        ### add processed and composite data array to lsit



    ### concatenates composite data arrays for each band along band dimension



In [ ]:
### call function to get final composite reflectance data 


## STEP 4: K-means clustering

Cluster your data by spectral signature using the k-means algorithm.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Convert your DataArray into a <strong>tidy</strong> DataFrame of
reflectance values (hint: check out the <code>.to_dataframe()</code> and
<code>.unstack()</code> methods)</li>
<li>Filter out all rows with no data (all 0s or any N/A values)</li>
<li>Fit a k-means model. You can experiment with the number of groups to
find what works best.</li>
</ol></div></div>

In [ ]:
### Convert spectral DataArray to a tidy DataFrame

### filter out rows with no data

Now we're reading to fit the k-means clustering model. We can run the fit and prediction functions at the same time because we don't have target data.

In [ ]:
### initialize k-means model 


### fit model and predict


### add the predicted values back to the model dataframe

## STEP 5: Plot

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Create a plot that shows the k-means clusters next to an RGB image of
the area. You may need to brighten your RGB image by multiplying it by
10. The code for reshaping and plotting the clusters is provided for you
below, but you will have to create the RGB plot yourself!</p>
<p>So, what is <code>.sortby(['x', 'y'])</code> doing for us? Try the
code without it and find out.</p></div></div>

In [ ]:
### make data array with bands to use for rgb: red, green, and blue


In [15]:
### plot the k-means clusters
(
    rgb_plot
    + 
    model_df.clusters.to_xarray().sortby(['x', 'y']).hvplot(
        cmap="Colorblind", aspect='equal') 
)

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Don’t forget to interpret your plot!</p></div></div>

**YOUR PLOT HEADLINE AND DESCRIPTION HERE**